In [1]:
from cryptofeed.defines import TRADES, L2_BOOK, BID, ASK

import pandas as pd
import json

import time
import datetime

import threading
import schedule
import redis

from algos.altcoin.backtest import perform_backtests
from algos.altcoin.live_trader import liveTrading, round_down
from utils import print

In [3]:
def get_positions():
    r = redis.Redis(host='localhost', port=6379, db=0)
    config = pd.read_csv('algos/altcoin/config.csv')
    details_df = pd.DataFrame()

    for idx, row in config.iterrows():
        asset = row['name']

        curr_detail = pd.Series()
        curr_detail['name'] = asset
        backtest = pd.read_csv("data/trades_{}.csv".format(asset))

        try:
            curr_detail['position'] = r.get('{}_current_pos'.format(asset)).decode()
        except:
            curr_detail['position'] = "NONE"

        try:
            curr_detail['entry'] = round(float(r.get('{}_avgEntryPrice'.format(asset)).decode()),2)
        except:
            curr_detail['entry'] = 0

        try:
            curr_detail['pos_size'] = float(r.get('{}_pos_size'.format(asset)).decode())
        except:
            curr_detail['pos_size'] = 0

        try:
            curr_detail['live_price'] = round(float(r.get('{}_best_ask'.format(asset)).decode()), 3)
        except:
            curr_detail['live_price'] = 0


        curr_detail['backtest_position'] = 'SHORT' if backtest.iloc[-1]['Type'] == 'SELL' else 'LONG'
        curr_detail['backtest_date'] = backtest.iloc[-1]['Date']
        curr_detail['entry_price'] = round(backtest.iloc[-1]['Price'], 2)
        curr_detail['to_trade'] = row['to_trade']
        curr_detail['live_lev'] = int(row['mult'])

        try:
            curr_detail['live_pnl'] = round(((curr_detail['live_price'] - curr_detail['entry'])/curr_detail['entry']) * 100 * curr_detail['live_lev'], 2)

            if curr_detail['position'] == 'SHORT':
                curr_detail['live_pnl'] = curr_detail['live_pnl'] * -1
        except:
            curr_detail['live_pnl'] = 0


        try:
            curr_detail['ftx_balance'] = round(float(r.get('{}_net_worth'.format(asset)).decode()),2)
        except:
            curr_detail['ftx_balance'] = 0
            
        try:
            curr_detail['backtest_pnl'] = round(((curr_detail['live_price'] - curr_detail['entry_price'])/curr_detail['entry_price']) * 100 * curr_detail['live_lev'], 2)

            if curr_detail['backtest_position'] == 'SHORT':
                curr_detail['backtest_pnl'] = curr_detail['backtest_pnl'] * -1
        except:
            curr_detail['backtest_pnl'] = 0
            
        curr_detail['allocation'] = row['allocation']

        details_df = details_df.append(curr_detail, ignore_index=True)
    
    return details_df

In [4]:
config = pd.read_csv('algos/altcoin/config.csv')

for pair in config['name']:
    lt = liveTrading(pair)
    lt.set_position()


enabled = 1

try:
    enabled = float(r.get('altcoin_enabled').decode())
except:
    pass


In [5]:
details_df = get_positions()

details_df['target_pos'] = details_df['backtest_position'].replace("LONG", 1).replace("SHORT", -1).replace("NONE", 0)
details_df['curr_pos'] = details_df['position'].replace("LONG", 1).replace("SHORT", -1).replace("NONE", 0)

to_close = details_df[details_df['target_pos'] == 0]

for idx, row in to_close.iterrows():
    if row['to_trade'] == 1:
        if row['curr_pos'] != 0:
            lt = liveTrading(symbol=row['name'])
            lt.fill_order('close', row['position'].lower())

to_open = details_df[details_df['target_pos'] != 0]

<ipython-input-3-84117639522a>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  curr_detail = pd.Series()


In [14]:
as_list = list(to_open.T.to_dict().values())

In [15]:
as_list

[{'allocation': 0.2,
  'backtest_date': '2021-01-29',
  'backtest_pnl': -3.53,
  'backtest_position': 'SHORT',
  'entry': 0.35,
  'entry_price': 0.34,
  'ftx_balance': 591.2,
  'live_lev': 2.0,
  'live_pnl': 2.29,
  'live_price': 0.346,
  'name': 'ADA-PERP',
  'pos_size': 613.0,
  'position': 'SHORT',
  'to_trade': 1.0,
  'target_pos': -1,
  'curr_pos': -1},
 {'allocation': 0.034,
  'backtest_date': '2021-01-18',
  'backtest_pnl': 80.01,
  'backtest_position': 'LONG',
  'entry': 19.38,
  'entry_price': 20.81,
  'ftx_balance': 52.53,
  'live_lev': 3.0,
  'live_pnl': 108.05,
  'live_price': 26.36,
  'name': 'BAL-PERP',
  'pos_size': 8.68,
  'position': 'LONG',
  'to_trade': 1.0,
  'target_pos': 1,
  'curr_pos': 1},
 {'allocation': 0.005,
  'backtest_date': '2021-02-01',
  'backtest_pnl': -7.22,
  'backtest_position': 'SHORT',
  'entry': 172.87,
  'entry_price': 174.1,
  'ftx_balance': 10.34,
  'live_lev': 3.0,
  'live_pnl': -9.41,
  'live_price': 178.29,
  'name': 'BSV-PERP',
  'pos_size